In [1]:
import pandas as pd
import re
import os

def csi_txt_to_excel(txt_path, out_dir=None, expand_data=False, max_data_cols=0):
    """
    Convert concatenated CSI text into an Excel .xlsx file.

    Parameters
    ----------
    txt_path : str
        Path to the input .txt file.
    out_dir : str or None
        Writable directory for the output .xlsx. If None, the function auto-selects:
        - same directory as txt_path if writable
        - '/kaggle/working' if available and writable
        - current working directory otherwise
    expand_data : bool
        If True, expand the values inside data=[...] into separate columns (Data0, Data1, ...).
        If False, keep the full list as a single string in column 'Data'.
    max_data_cols : int
        Max number of data columns to create when expand_data=True.
        0 means auto-cap to Excel's column limit (up to 16,384 minus metadata columns).

    Returns
    -------
    str
        Absolute path to the saved .xlsx file.
    """

    # Read the entire file
    with open(txt_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Normalize escaped newlines in case the file is one long line like "...]\nCSI_DATA..."
    content = content.replace("\\n", "\n")

    # Split into records; keep 'CSI_DATA,' at the start of each chunk
    chunks = re.split(r'(?=CSI_DATA,)', content)
    rows = []

    for rec in chunks:
        rec = rec.strip()
        if not rec.startswith("CSI_DATA,"):
            continue

        # Extract MAC: token right after "CSI_DATA,"
        m_mac = re.search(r'^CSI_DATA,([^,]+)', rec)
        mac = m_mac.group(1).strip() if m_mac else ""

        # Extract timestamp after "Now="
        m_time = re.search(r'Now=\s*([0-9\-]{10}\s[0-9:\.]{8,})', rec)
        timestamp = m_time.group(1).strip() if m_time else ""

        # Extract data list inside brackets
        m_data = re.search(r'data=\[([^\]]*)\]', rec)
        data_str = m_data.group(1).strip() if m_data else ""
        data_list = [s.strip() for s in data_str.split(",")] if data_str else []

        row = {"MAC": mac, "Timestamp": timestamp, "Data": data_str}

        if expand_data:
            # Determine max columns we can safely create for Excel
            if max_data_cols <= 0:
                # Excel limit is 16,384 columns; we already have 3 meta columns
                max_cols_possible = 16384 - 3
            else:
                max_cols_possible = max_data_cols

            for i, val in enumerate(data_list[:max_cols_possible]):
                row[f"Data{i}"] = val

            # Inform if truncated
            if len(data_list) > max_cols_possible:
                row["Data_Truncated"] = f"{len(data_list) - max_cols_possible} values truncated"

        rows.append(row)

    # Build DataFrame
    df = pd.DataFrame(rows)

    # Helper to test if a directory is writable
    def is_writable(dirpath: str) -> bool:
        try:
            test_path = os.path.join(dirpath, ".write_test")
            with open(test_path, "w") as tmp:
                tmp.write("ok")
            os.remove(test_path)
            return True
        except Exception:
            return False

    # Decide output directory
    src_dir = os.path.dirname(os.path.abspath(txt_path))
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
        target_dir = out_dir
    elif is_writable(src_dir):
        target_dir = src_dir
    elif os.path.isdir("/kaggle/working") and is_writable("/kaggle/working"):
        target_dir = "/kaggle/working"
    else:
        target_dir = os.getcwd()

    # Build output path
    base_name = os.path.splitext(os.path.basename(txt_path))[0]
    excel_path = os.path.join(target_dir, base_name + ".xlsx")

    # Save to Excel
    df.to_excel(excel_path, index=False)

    return os.path.abspath(excel_path)


# Example usage (auto-saves to /kaggle/working since /kaggle/input is read-only):
paths = ['csi1/15-41-51-299.txt']
for i in paths:
    out_path = csi_txt_to_excel(i)




FileNotFoundError: [Errno 2] No such file or directory: 'csi1/15-41-51-299.txt'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os

paths = ['csi1/15-41-51-299']
for inumber,i in enumerate(paths):
    file_path = f"{i}.xlsx"
    df = pd.read_excel(file_path)

    print("Shape:", df.shape)
    print(df.head())

    # ===== 2. پردازش ستون Data =====
    # جدا کردن رشته به لیست اعداد
    df["Data_list"] = df["Data"].apply(lambda x: np.array([int(i) for i in str(x).split(",") if i.strip() != ""]))

    # پیدا کردن طول بیشینه
    max_len = df["Data_list"].apply(len).max()

    # نرمال کردن طول آرایه‌ها (padding با NaN)
    data_matrix = np.array([
        np.pad(arr.astype(float), (0, max_len - len(arr)), constant_values=np.nan)
        for arr in df["Data_list"]
    ])

    # ساخت دیتافریم عددی
    df_expanded = pd.DataFrame(data_matrix)

    print("\nExpanded shape:", df_expanded.shape)

    # ===== 3. محاسبات آماری =====
    stats = df_expanded.describe().T
    stats["variance"] = df_expanded.var(skipna=True)
    print("\n=== Statistical Summary (first 10 features) ===\n", stats.head(10))

    # ===== 4. شناسایی Outliers =====
    z_scores = np.abs(zscore(df_expanded, nan_policy='omit'))
    outliers = (z_scores > 3).sum(axis=0)
    print("\nNumber of outliers in each feature (first 10 cols):\n", outliers[:10])

    # ===== 5. ترسیم نمودار برای هر کانال (با 3 ساب‌پلات) =====
    output_dir = "plots_all_channels"
    os.makedirs(output_dir, exist_ok=True)

    for col in df_expanded.columns:
        channel_data = df_expanded[col].dropna()

        fig, axes = plt.subplots(1, 3, figsize=(18, 4))

        # --- Histogram ---
        axes[0].hist(channel_data, bins=30, color="skyblue", edgecolor="black")
        axes[0].set_title(f"Histogram - Channel {col}")
        axes[0].set_xlabel("Value")
        axes[0].set_ylabel("Frequency")

        # --- Boxplot ---
        sns.boxplot(x=channel_data, ax=axes[1], color="lightcoral")
        axes[1].set_title(f"Boxplot - Channel {col}")
        axes[1].set_xlabel("Value")

        # --- Line plot ---
        axes[2].plot(channel_data, color="navy")
        axes[2].set_title(f"Time Series - Channel {col}")
        axes[2].set_xlabel("Index")
        axes[2].set_ylabel("Value")

        plt.suptitle(f"Channel {col} - Data Analysis", fontsize=14)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(os.path.join(output_dir, f"channel_{col}_{inumber}_analysis.png"))
        print(inumber)
        plt.close()

    print(f"\n✅ All combined plots saved in folder: {output_dir}")


FileNotFoundError: [Errno 2] No such file or directory: 'csi1/15-41-51-299.xlsx'

In [3]:
import pandas as pd
import re
import os
from tqdm import tqdm

def csi_txt_to_excel(txt_path, out_dir=None, expand_data=False, max_data_cols=0):
    """
    Convert concatenated CSI text into an Excel .xlsx file.

    Parameters
    ----------
    txt_path : str
        Path to the input .txt file.
    out_dir : str or None
        Writable directory for the output .xlsx. If None, the function auto-selects:
        - same directory as txt_path if writable
        - '/kaggle/working' if available and writable
        - current working directory otherwise
    expand_data : bool
        If True, expand the values inside data=[...] into separate columns (Data0, Data1, ...).
        If False, keep the full list as a single string in column 'Data'.
    max_data_cols : int
        Max number of data columns to create when expand_data=True.
        0 means auto-cap to Excel's column limit (up to 16,384 minus metadata columns).

    Returns
    -------
    str
        Absolute path to the saved .xlsx file.
    """

    # Read the entire file
    with open(txt_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Normalize escaped newlines in case the file is one long line like "...]\nCSI_DATA..."
    content = content.replace("\\n", "\n")

    # Split into records; keep 'CSI_DATA,' at the start of each chunk
    chunks = re.split(r'(?=CSI_DATA,)', content)
    rows = []

    for rec in chunks:
        rec = rec.strip()
        if not rec.startswith("CSI_DATA,"):
            continue

        # Extract MAC: token right after "CSI_DATA,"
        m_mac = re.search(r'^CSI_DATA,([^,]+)', rec)
        mac = m_mac.group(1).strip() if m_mac else ""

        # Extract timestamp after "Now="
        m_time = re.search(r'Now=\s*([0-9\-]{10}\s[0-9:\.]{8,})', rec)
        timestamp = m_time.group(1).strip() if m_time else ""

        # Extract data list inside brackets
        m_data = re.search(r'data=\[([^\]]*)\]', rec)
        data_str = m_data.group(1).strip() if m_data else ""
        data_list = [s.strip() for s in data_str.split(",")] if data_str else []

        row = {"MAC": mac, "Timestamp": timestamp, "Data": data_str}

        if expand_data:
            # Determine max columns we can safely create for Excel
            if max_data_cols <= 0:
                # Excel limit is 16,384 columns; we already have 3 meta columns
                max_cols_possible = 16384 - 3
            else:
                max_cols_possible = max_data_cols

            for i, val in enumerate(data_list[:max_cols_possible]):
                row[f"Data{i}"] = val

            # Inform if truncated
            if len(data_list) > max_cols_possible:
                row["Data_Truncated"] = f"{len(data_list) - max_cols_possible} values truncated"

        rows.append(row)

    # Build DataFrame
    df = pd.DataFrame(rows)

    # Helper to test if a directory is writable
    def is_writable(dirpath: str) -> bool:
        try:
            test_path = os.path.join(dirpath, ".write_test")
            with open(test_path, "w") as tmp:
                tmp.write("ok")
            os.remove(test_path)
            return True
        except Exception:
            return False

    # Decide output directory
    src_dir = os.path.dirname(os.path.abspath(txt_path))
    if out_dir:
        os.makedirs(out_dir, exist_ok=True)
        target_dir = out_dir
    elif is_writable(src_dir):
        target_dir = src_dir
    elif os.path.isdir("/kaggle/working") and is_writable("/kaggle/working"):
        target_dir = "/kaggle/working"
    else:
        target_dir = os.getcwd()

    # Build output path
    base_name = os.path.splitext(os.path.basename(txt_path))[0]
    excel_path = os.path.join(target_dir, 'xl')
    excel_path = os.path.join(excel_path, base_name + ".xlsx")

    # Save to Excel
    df.to_excel(excel_path, index=False)
    return os.path.abspath(excel_path)

def convert_all_txt_in_folders(root_folders, expand_data=False, max_data_cols=0):
    """
    Convert all .txt files inside given folders to .xlsx files using csi_txt_to_excel,
    saving each Excel file in the same folder as the original .txt file.

    Parameters
    ----------
    root_folders : list of str
        List of folder paths containing .txt files to convert.
    expand_data : bool
        Passed to csi_txt_to_excel to control data expansion.
    max_data_cols : int
        Passed to csi_txt_to_excel for max data columns if expand_data=True.
    """
    for folder in tqdm(root_folders):
        for dirpath, _, filenames in os.walk(folder):
            for filename in filenames:
                if filename.lower().endswith('.txt'):
                    txt_path = os.path.join(dirpath, filename)
                    # print(f"Converting: {txt_path}")
                    out_path = csi_txt_to_excel(txt_path, out_dir=dirpath,
                                                expand_data=expand_data,
                                                max_data_cols=max_data_cols)
                    # print(f"✅ Saved to: {out_path}")

folders = [
    r"D:\Mojtaba\Dataset_Master_minds\DataSet_test\csi1",
    r"D:\Mojtaba\Dataset_Master_minds\DataSet_test\csi2",
    r"D:\Mojtaba\Dataset_Master_minds\DataSet_test\csi3",
    r"D:\Mojtaba\Dataset_Master_minds\DataSet_test\csi4"
]

convert_all_txt_in_folders(folders)

100%|██████████| 4/4 [00:00<00:00, 7446.61it/s]


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os
from tqdm import tqdm

folders = [
    r"DataSet_test\csi1\xl",
    r"DataSet_test\csi2\xl",
    r"DataSet_test\csi3\xl",
    r"DataSet_test\csi4\xl"
]

for folder in tqdm(folders, desc="Folders", unit="folder"):
    # پیدا کردن همه فایل‌های xlsx داخل این فولدر (و زیر‌فولدرها)
    xlsx_paths = []
    for dirpath, _, filenames in os.walk(folder):
        for fn in filenames:
            if fn.lower().endswith('.xlsx'):
                xlsx_paths.append(os.path.join(dirpath, fn))

    if len(xlsx_paths) == 0:
        tqdm.write(f"No .xlsx files found in: {folder}")
        continue

    # فقط 1 نمونه (فایل) از هر پوشه بردار
    xlsx_paths = xlsx_paths[:1]

    # نوار پیشرفت برای فایل‌های انتخاب‌شده
    for file_idx, xlsx_path in enumerate(tqdm(xlsx_paths, desc=f"Files in {os.path.basename(folder)}", leave=False, unit="file")):
        filename = os.path.basename(xlsx_path)
        dirpath = os.path.dirname(xlsx_path)

        df = pd.read_excel(xlsx_path)

        # ===== 2. پردازش ستون Data =====
        df["Data_list"] = df["Data"].apply(lambda x: np.array([int(i) for i in str(x).split(",") if i.strip() != ""]))

        max_len = df["Data_list"].apply(len).max()

        data_matrix = np.array([
            np.pad(arr.astype(float), (0, max_len - len(arr)), constant_values=np.nan)
            for arr in df["Data_list"]
        ])

        df_expanded = pd.DataFrame(data_matrix)

        # ===== 3. محاسبات آماری =====
        stats = df_expanded.describe().T
        stats["variance"] = df_expanded.var(skipna=True)

        # ===== 4. شناسایی Outliers =====
        z_scores = np.abs(zscore(df_expanded, nan_policy='omit'))
        outliers = (z_scores > 3).sum(axis=0)

        # ===== 5. ترسیم نمودار =====
        output_dir = os.path.join(dirpath, 'plots')
        os.makedirs(output_dir, exist_ok=True)

        # تعداد جفت کانال = ceil(total_channels / 2)
        n_pairs = (df_expanded.shape[1] + 1) // 2

        # tqdm برای هر جفت کانال — با total قابل نمایش ETA
        for pair_num in tqdm(range(n_pairs),
                             desc=f"File {file_idx+1}/{len(xlsx_paths)}: {filename}",
                             leave=False,
                             unit="pair"):
            i = pair_num * 2  # کانال real شروع
            fig, axes = plt.subplots(2, 3, figsize=(18, 8))

            for j in range(2):  # 0 = real, 1 = imag
                col = i + j
                if col >= df_expanded.shape[1]:
                    # اگر تعداد کانال‌ها فرد بود ممکنه اینجا از حدود بیرون بزنه؛ ادامه بده
                    for empty_ax in axes[j]:
                        empty_ax.axis('off')
                    continue

                channel_data = df_expanded[col].dropna()

                # اگر داده‌ای نبود، محتوای خالی بذار
                if channel_data.size == 0:
                    for ax in axes[j]:
                        ax.text(0.5, 0.5, "No data", ha='center', va='center', transform=ax.transAxes)
                        ax.set_axis_off()
                    continue

                mean_val = stats.loc[col, "mean"]
                var_val = stats.loc[col, "variance"]

                # --- Histogram ---
                axes[j, 0].hist(channel_data, bins=30, edgecolor="black")
                axes[j, 0].axvline(mean_val, color="red", linestyle="--", label=f"Mean={mean_val:.2f}")
                axes[j, 0].legend()
                axes[j, 0].set_title(f"Histogram - Channel {col}\nVariance={var_val:.2f}")
                axes[j, 0].set_xlabel("Value")
                axes[j, 0].set_ylabel("Frequency")

                # --- Boxplot ---
                sns.boxplot(x=channel_data, ax=axes[j, 1])
                # اگر outliers[col] NaN بود، نمایش نده
                try:
                    out_count = int(outliers[col])
                except Exception:
                    out_count = 0
                axes[j, 1].set_title(f"Boxplot - Channel {col}\nOutliers={out_count}")
                axes[j, 1].set_xlabel("Value")

                # --- Line plot ---
                axes[j, 2].plot(channel_data)
                axes[j, 2].set_title(f"Time Series - Channel {col}")
                axes[j, 2].set_xlabel("Index")
                axes[j, 2].set_ylabel("Value")
                axes[j, 2].text(0.01, 0.95,
                                f"Mean={mean_val:.2f}\nMin={channel_data.min():.2f}\nMax={channel_data.max():.2f}",
                                transform=axes[j, 2].transAxes,
                                fontsize=9, verticalalignment='top',
                                bbox=dict(facecolor='white', alpha=0.6))

            plt.suptitle(f"Channels {i} (Real) & {i+1} (Imag) - Data Analysis", fontsize=14)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            out_fname = os.path.join(output_dir, f"channels_{i}_{i+1}_analysis.png")
            plt.savefig(out_fname)
            plt.close()


Files in xl:   0%|          | 0/1 [00:00<?, ?file/s]

File 1/1: 15-41-51-299.xlsx:   0%|          | 0/64 [00:00<?, ?pair/s]

File 1/1: 15-41-51-299.xlsx:   2%|▏         | 1/64 [00:01<01:04,  1.02s/pair]

File 1/1: 15-41-51-299.xlsx:   3%|▎         | 2/64 [00:01<00:58,  1.06pair/s]

File 1/1: 15-41-51-299.xlsx:   5%|▍         | 3/64 [00:02<00:55,  1.11pair/s]